#### **Finite Element Analysis of Thermal Expansion and Stress in Machine Parts**
#### **Purpose:** This project develops a Python-based numerical solver to evaluate how heat from machining affects solid metal components. By dividing the part into Constant Strain Triangle (CST) elements, the study simulates how heat flux causes physical deformation and internal stress. The analysis combines machining physics with solid mechanics to identify critical stress points and compares how different materials (Aluminum, Steel, and Titanium) perform in high-precision manufacturing environments.

##### **Author:** Bello Oluwatobi

##### **Last Updated:** August 19, 2025

### #1 Importing the libraries

In [ ]:
# importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt

### #2 Defining the study parameters

In [ ]:
# defining the machining power for a roughing operation
CUTTING_POWER = 4000.0  # Watts

# defining the heat generated that flows into the workpiece
HEAT_PARTITION = 0.10 # 10%

# defining the area where the tool engages with the workpiece
CONTACT_AREA = 0.002  # m2

# calculating the heat flux (q)
HEAT_FLUX_Q = (CUTTING_POWER * HEAT_PARTITION) / CONTACT_AREA

print(f"Calculated Heat Flux (q): {HEAT_FLUX_Q} W/m^2")

In [ ]:
# defining the machine part dimensions (in meters)
LENGTH = 0.2
HEIGHT = 0.1
THICKNESS = 0.01

# defining the mesh grid density (12x6 grid)
NUM_ELEMENTS_X = 12
NUM_ELEMENTS_Y = 6

In [ ]:
# initilizing the materials dictionary
materials = {}

# defining material 1: Aluminum 6061
materials["Aluminum 6061"] = {
    "E": 68.9e9,          # Young's Modulus (Pa)
    "nu": 0.33,         # Poisson's Ratio
    "alpha": 23.2e-6,     # Thermal Expansion Coefficient (1/C)
    "k": 167.0,         # Thermal Conductivity (W/mK)
    "yield": 276e6      # Yield Strength (Pa)
}

# defining material 2: Structural Steel (A36)
materials["Structural Steel"] = {
    "E": 207e9,
    "nu": 0.3,
    "alpha": 11.7e-6,
    "k": 51.9,
    "yield": 220e6
}

# defining material 3: Titanium Ti-6Al-4V
materials["Titanium (Ti-6Al-4V)"] = {
    "E": 114e9,
    "nu": 0.34,
    "alpha": 8.6e-6,
    "k": 7.2,
    "yield": 910e6
}

### #3 Generating Mesh

In [ ]:
# creating linear arrays for X and Y coordinates
x_coordinates = np.linspace(0, LENGTH, NUM_ELEMENTS_X + 1)
y_coordinates = np.linspace(0, HEIGHT, NUM_ELEMENTS_Y + 1)

# creating the mesh grid
X_GRID, Y_GRID = np.meshgrid(x_coordinates, y_coordinates)

# flattening the arrays to create the node list i.e. [ [x0, y0], [x1, y1], ... ]
nodes = np.stack([X_GRID.flatten(), Y_GRID.flatten()], axis=1)
total_nodes = len(nodes)

# generating the elements i.e. split rectangular grid cells into two triangles
elements = []

for j in range(NUM_ELEMENTS_Y):
    for i in range(NUM_ELEMENTS_X):
        # calculating node indices for the current rectangle
        n0 = j * (NUM_ELEMENTS_X + 1) + i
        n1 = n0 + 1
        n2 = n0 + (NUM_ELEMENTS_X + 1)
        n3 = n2 + 1
        
        # setting triangle 1: Nodes [n0, n1, n3]
        elements.append([n0, n1, n3])
        
        # setting triangle 2: Nodes [n0, n3, n2]
        elements.append([n0, n3, n2])

# getting the number of nodes and elements generated
elements = np.array(elements)
total_elements = len(elements)

print(f"Mesh Generated: {total_nodes} Nodes, {total_elements} Elements")

### #4 Running the simulation

In [ ]:
# setting up the plotting figure
fig, axes = plt.subplots(3, 2, figsize=(16, 14))
plt.subplots_adjust(hspace=0.4, wspace=0.2)

# displaying table header (for results)
print("-" * 85)
print(f"{'Material':<20} | {'Max Temp (C)':<15} | {'Max Stress (MPa)':<18} | {'Safety Factor':<15}")
print("-" * 85)

# executing the simulation loop
for idx, (material_name, props) in enumerate(materials.items()):
    
    # calculating the thermal field
    conductivity = props["k"]
    calculated_delta_T = (HEAT_FLUX_Q * LENGTH) / conductivity
    
    if calculated_delta_T > 400.0:
        calculated_delta_T = 400.0
        
    node_temperatures = (nodes[:, 0] / LENGTH) * calculated_delta_T
    
    # visualizing the temperature field
    current_ax_temp = axes[idx, 0]
    
    # plotting the temperature contour
    tp = current_ax_temp.tripcolor(
        nodes[:, 0], 
        nodes[:, 1], 
        elements, 
        node_temperatures, 
        cmap='coolwarm', 
        shading='gouraud', 
        edgecolors='k', 
        linewidth=0.0
    )

    # overlaying the contour with mesh
    current_ax_temp.triplot(
        nodes[:, 0], 
        nodes[:, 1], 
        elements, 
        'k-',          
        linewidth=0.5,
        alpha=0.4   
    )

    # overlaying the nodes
    current_ax_temp.plot(nodes[:,0], nodes[:,1], 'ko', markersize=2)
    
    current_ax_temp.set_title(f"{material_name}\nApplied Thermal Field (k={conductivity} W/mK)")
    current_ax_temp.set_xlabel("Length (m)")
    current_ax_temp.set_ylabel("Height (m)")
    current_ax_temp.axis('equal')
    
    # adding the colorbar
    cbar1 = fig.colorbar(tp, ax=current_ax_temp)
    cbar1.set_label("Temperature (C)", rotation=270, labelpad=15)

    # executing the finite element solver
    num_dofs = 2 * total_nodes
    K_global = np.zeros((num_dofs, num_dofs))
    F_thermal = np.zeros(num_dofs)
    
    E = props["E"]
    nu = props["nu"]
    alpha = props["alpha"]
    
    prefactor = E / (1 - nu**2)
    D = prefactor * np.array([[1, nu, 0], [nu, 1, 0], [0, 0, (1-nu)/2]])
    
    for e_idx in range(total_elements):
        node_indices = elements[e_idx]
        coords = nodes[node_indices]
        
        # setting up the geometry
        x1, y1 = coords[0]; x2, y2 = coords[1]; x3, y3 = coords[2]
        matrix_for_area = np.array([[1, x1, y1],[1, x2, y2],[1, x3, y3]])
        Area_e = 0.5 * abs(np.linalg.det(matrix_for_area))
        
        # setting up the B-Matrix
        b1, b2, b3 = y2-y3, y3-y1, y1-y2
        c1, c2, c3 = x3-x2, x1-x3, x2-x1
        B = (1 / (2 * Area_e)) * np.array([[b1,0,b2,0,b3,0],[0,c1,0,c2,0,c3],[c1,b1,c2,b2,c3,b3]])
        
        # setting up the load
        avg_x = (x1 + x2 + x3) / 3.0
        local_temp_rise = (avg_x / LENGTH) * calculated_delta_T
        thermal_strain = np.array([alpha * local_temp_rise, alpha * local_temp_rise, 0])
        
        k_e = (B.T @ D @ B) * Area_e * THICKNESS
        f_e = (B.T @ D @ thermal_strain) * Area_e * THICKNESS
        
        # setting the stiffness assembly
        element_dofs = []
        for node_id in node_indices:
            element_dofs.append(2 * node_id)
            element_dofs.append(2 * node_id + 1)
            
        for row in range(6):
            global_row = element_dofs[row]
            F_thermal[global_row] += f_e[row]
            for col in range(6):
                K_global[global_row, element_dofs[col]] += k_e[row, col]
                
    # setting the boundary conditions and solving the linear set of equations
    fixed_dofs = []
    for i in range(total_nodes):
        if nodes[i, 0] == 0.0:
            fixed_dofs.append(2 * i)
            fixed_dofs.append(2 * i + 1)
            
    all_dofs = np.arange(num_dofs)
    free_dofs = np.setdiff1d(all_dofs, fixed_dofs)
    
    U_global = np.zeros(num_dofs)
    U_global[free_dofs] = np.linalg.solve(K_global[np.ix_(free_dofs, free_dofs)], F_thermal[free_dofs])
    
    # running the post processing step i.e. calculating Von-Mises stress
    von_mises_stresses = []
    for e_idx in range(total_elements):
        node_indices = elements[e_idx]
        coords = nodes[node_indices]
        
        # recalculating the geometry
        x1, y1 = coords[0]; x2, y2 = coords[1]; x3, y3 = coords[2]
        matrix_for_area = np.array([[1, x1, y1],[1, x2, y2],[1, x3, y3]])
        Area_e = 0.5 * abs(np.linalg.det(matrix_for_area))
        b1, b2, b3 = y2-y3, y3-y1, y1-y2; c1, c2, c3 = x3-x2, x1-x3, x2-x1
        B = (1 / (2 * Area_e)) * np.array([[b1,0,b2,0,b3,0],[0,c1,0,c2,0,c3],[c1,b1,c2,b2,c3,b3]])
        
        # calculating the displacements
        element_dofs = []
        for node_id in node_indices:
            element_dofs.append(2 * node_id); element_dofs.append(2 * node_id + 1)
        u_element = U_global[element_dofs]
        
        # calculating the stress
        avg_x = (x1 + x2 + x3) / 3.0
        local_temp = (avg_x / LENGTH) * calculated_delta_T
        strain_thermal = np.array([alpha * local_temp, alpha * local_temp, 0])
        sigma = D @ (B @ u_element - strain_thermal)
        vm = np.sqrt(sigma[0]**2 + sigma[1]**2 - sigma[0]*sigma[1] + 3*sigma[2]**2)
        von_mises_stresses.append(vm)

    
    # visualizing the stress results
    current_ax_stress = axes[idx, 1]
    SCALE_FACTOR = 200 # making the deformation more visible
    deformed_nodes = nodes + U_global.reshape(-1, 2) * SCALE_FACTOR
    
    tc_stress = current_ax_stress.tripcolor(
        deformed_nodes[:, 0], 
        deformed_nodes[:, 1], 
        elements, 
        facecolors=np.array(von_mises_stresses), 
        cmap='viridis', 
        edgecolors='k', 
        linewidth=0.1
    )
    
    # overlaying the deformed mesh wireframe
    current_ax_stress.triplot(deformed_nodes[:, 0], deformed_nodes[:, 1], elements, 'k-', lw=0.2, alpha=0.3)
    
    max_vm = max(von_mises_stresses)
    yield_strength = props["yield"]
    safety_factor = yield_strength / max_vm
    
    current_ax_stress.set_title(f"Von Mises Stress (Scale {SCALE_FACTOR}x)\nMax: {max_vm/1e6:.1f} MPa | SF: {safety_factor:.2f}")
    current_ax_stress.set_xlabel("Length (m)")
    current_ax_stress.set_ylabel("Height (m)")
    current_ax_stress.axis('equal')
    
    cbar2 = fig.colorbar(tc_stress, ax=current_ax_stress)
    cbar2.set_label("Von Mises Stress (Pa)", rotation=270, labelpad=15)
    
    # displaying results
    print(f"{material_name:<20} | {calculated_delta_T:<15.1f} | {max_vm/1e6:<18.2f} | {safety_factor:<15.2f}")

# displaying the graphs
plt.savefig('../results/material_comparison.png', dpi=300, bbox_inches='tight')
plt.show()